In [1]:
from utils import get_tickers_from_market_index

idiv = get_tickers_from_market_index("./assets/IDIV-composition.csv")

len(idiv)

51

In [3]:
tickers_code = [f"{ticker}.SA" for ticker in idiv]
tickers_code

['ABCB4.SA',
 'ALUP11.SA',
 'AURE3.SA',
 'B3SA3.SA',
 'BRSR6.SA',
 'BBSE3.SA',
 'BRAP4.SA',
 'BBAS3.SA',
 'AGRO3.SA',
 'CXSE3.SA',
 'CMIG3.SA',
 'CMIG4.SA',
 'CSMG3.SA',
 'CPLE3.SA',
 'CPLE6.SA',
 'CPFE3.SA',
 'CMIN3.SA',
 'DIRR3.SA',
 'EGIE3.SA',
 'FESA4.SA',
 'FLRY3.SA',
 'GGBR4.SA',
 'GOAU4.SA',
 'GRND3.SA',
 'RANI3.SA',
 'ITSA4.SA',
 'JBSS3.SA',
 'JHSF3.SA',
 'KEPL3.SA',
 'KLBN11.SA',
 'LAVV3.SA',
 'MRFG3.SA',
 'LEVE3.SA',
 'BEEF3.SA',
 'PETR3.SA',
 'PETR4.SA',
 'PSSA3.SA',
 'RAPT4.SA',
 'ROMI3.SA',
 'SANB11.SA',
 'CSNA3.SA',
 'TAEE11.SA',
 'TASA4.SA',
 'TGMA3.SA',
 'VIVT3.SA',
 'TRPL4.SA',
 'TRIS3.SA',
 'UNIP6.SA',
 'USIM5.SA',
 'VALE3.SA',
 'WIZC3.SA']

## Obter os dados de dividendos

In [4]:
import yfinance as yf

tickers = yf.Tickers(tickers_code).tickers

In [5]:
import pandas as pd

all_dividends = pd.DataFrame()

for ticker, data in tickers.items():
    per_month_df = data.actions.groupby(pd.Grouper(freq="M")).sum()
    dividend_df = per_month_df.iloc[:, 0].to_frame()
    dividend_df.rename(columns={'Dividends': ticker}, inplace=True)

    if all_dividends.empty:
        all_dividends = dividend_df
        continue
    
    all_dividends = all_dividends.merge(dividend_df, left_index=True, right_index=True, how='outer')
    all_dividends.fillna(0, inplace=True)

all_dividends.reset_index(inplace=True)
all_dividends.set_index("Date", inplace=True)
all_dividends.index = pd.to_datetime(all_dividends.index)

all_dividends

,ABCB4.SA,ALUP11.SA,AURE3.SA,B3SA3.SA,BRSR6.SA,BBSE3.SA,BRAP4.SA,BBAS3.SA,AGRO3.SA,CXSE3.SA,...,TAEE11.SA,TASA4.SA,TGMA3.SA,VIVT3.SA,TRPL4.SA,TRIS3.SA,UNIP6.SA,USIM5.SA,VALE3.SA,WIZC3.SA
Date,,,,,,,,,,,,,,,,,,,,,
2000-03-31 00:00:00-03:00,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
2000-04-30 00:00:00-03:00,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
2000-05-31 00:00:00-03:00,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
2000-06-30 00:00:00-03:00,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
2000-07-31 00:00:00-03:00,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-30 00:00:00-03:00,0.000,0.00,0.0,0.115354,0.342607,0.000000,0.0,0.993051,0.0,0.0,...,0.000000,0.0,0.00,0.192782,0.0,0.0,0.0,0.0,0.000000,0.0
2023-07-31 00:00:00-03:00,0.836,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
2023-08-31 00:00:00-03:00,0.000,0.12,0.0,0.000000,0.000000,1.607637,0.0,0.668883,0.0,0.0,...,0.909844,0.1,0.57,0.244259,0.0,0.0,0.0,0.0,1.918472,0.0


In [6]:
all_dividends.to_csv("./assets/idiv_companies_dividends.csv")

In [9]:
import plotly.express as px

fig = px.line(all_dividends, x=all_dividends.index, y=all_dividends.columns[:5])
fig.update_layout(title="Dividendo das empresas listados em bolsa",
                  xaxis_title="Data",
                  yaxis_title="Valor")
fig.show()

/home/gabriel/Projects/3A-ai/.venv/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



## Previsão de dividendos

In [166]:
total_dividend = all_dividends.sum().to_frame("Dividend")
total_dividend

,Dividend
ABCB4.SA,9.904649
ALUP11.SA,9.446915
AURE3.SA,1.586049
B3SA3.SA,4.543908
BRSR6.SA,12.106016
BBSE3.SA,21.857815
BRAP4.SA,26.152799
BBAS3.SA,28.110513
AGRO3.SA,12.620133
CXSE3.SA,1.403551


In [165]:
px.box(total_dividend)

In [168]:
total_dividend.drop("CPLE6.SA", inplace=True)
total_dividend

,Dividend
ABCB4.SA,9.904649
ALUP11.SA,9.446915
AURE3.SA,1.586049
B3SA3.SA,4.543908
BRSR6.SA,12.106016
BBSE3.SA,21.857815
BRAP4.SA,26.152799
BBAS3.SA,28.110513
AGRO3.SA,12.620133
CXSE3.SA,1.403551


In [169]:
total_dividend.describe()

,Dividend
count,49.000000
mean,13.766041
std,11.814438
min,1.403551
25%,5.474693
50%,10.499968
75%,16.948375
max,48.908696


In [170]:
from sklearn.preprocessing import StandardScaler

scaller = StandardScaler()
total_dividend_scaled = scaller.fit_transform(total_dividend.to_numpy())
total_dividend_scaled

array([[-0.33022368],
       [-0.36936879],
       [-1.04162486],
       [-0.78867071],
       [-0.14196423],
       [ 0.69200319],
       [ 1.05930739],
       [ 1.22672979],
       [-0.09799728],
       [-1.05723197],
       [-0.07044586],
       [-0.21229362],
       [-0.27931239],
       [ 0.77609675],
       [-1.02005697],
       [-0.62702924],
       [ 1.30565568],
       [-0.04027592],
       [-0.51172787],
       [-0.00340947],
       [ 0.09678952],
       [-0.70907059],
       [-0.96338775],
       [-0.69477715],
       [-0.45877763],
       [-0.94549847],
       [-0.83031495],
       [-0.58825332],
       [-1.04234263],
       [-0.53558127],
       [ 0.27215112],
       [-0.91974706],
       [ 1.70918116],
       [ 2.01426098],
       [ 0.16591874],
       [-0.86712658],
       [-0.61917335],
       [ 0.61064565],
       [-0.01050766],
       [ 1.59856241],
       [-0.56713585],
       [-0.26638991],
       [ 3.00537668],
       [ 0.23976802],
       [-1.04397776],
       [ 1

In [185]:
from sklearn.cluster import KMeans

cluster_model = KMeans(n_clusters=3)
cluster_model.fit(total_dividend_scaled)

/home/gabriel/Projects/3A-ai/.venv/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



KMeans(n_clusters=3)

In [195]:
centers = cluster_model.cluster_centers_
centers = scaller.inverse_transform(centers).ravel()
centers

array([ 4.81058722, 14.72939565, 34.83252978])

In [196]:
import numpy as np

labels = cluster_model.labels_
np.unique(np.array(labels), return_counts=True)

(array([0, 1, 2], dtype=int32), array([23, 17,  9]))

In [202]:
import plotly.graph_objects as go

graph1 = px.scatter(total_dividend, x = total_dividend.index, y = total_dividend.columns, color=labels)
graph2 = px.scatter(x = ["Dividendos Baixo", "Dividendos Moderado", "Dividendos Alto"], y = centers, size=[12, 12, 12])
graph3 = go.Figure(data = graph1.data + graph2.data)
graph3.show()

In [205]:
total_dividend["Classification"] = labels
total_dividend

,Dividend,Classification
ABCB4.SA,9.904649,1
ALUP11.SA,9.446915,0
AURE3.SA,1.586049,0
B3SA3.SA,4.543908,0
BRSR6.SA,12.106016,1
BBSE3.SA,21.857815,1
BRAP4.SA,26.152799,2
BBAS3.SA,28.110513,2
AGRO3.SA,12.620133,1
CXSE3.SA,1.403551,0


In [71]:
from prophet import Prophet

model = Prophet()
model